In [1]:
import copy


def pauliMul(a, b):

    if a == b:
        return "I"
    if a == "I":
        return b
    if b == "I":
        return a
    ps = ["X", "Y", "Z"]
    ps.remove(a.upper())
    ps.remove(b.upper())
    return ps[0]


class pauliLog:

    def __init__(self, n, pauli, index):
        self.n = n
        self.pauli = pauli.upper()
        self.index = tuple(index)
        self.map = {index[i]: pauli[i].upper() for i in range(len(index))}

    def correct(self, grdStab):

        pass

    def merge(self, other):
        newMap = copy.deepcopy(self.map)
        for k in other.map.keys():
            if k in newMap.keys():
                res = pauliMul(newMap[k], other.map[k])
                if res == "I":
                    newMap.pop(k)
                else:
                    newMap[k] = res
            else:
                newMap[k] = other.map[k]
        pstr = ""
        index = []
        for k in newMap.keys():
            index.append(k)
            pstr += newMap[k]
        return pauliLog(self.n, pstr, index)

    def fit(self, other, commute):
        overlap = []
        res = 1
        for i in range(len(self.index)):
            ind1 = self.index[i]
            for j in range(len(other.index)):
                ind2 = other.index[j]
                if ind1 == ind2:
                    if other.pauli[j] != self.pauli[i]:
                        res *= -1

        return res == commute

    def __str__(self):
        return str((self.pauli.upper(), self.index))

    def __repr__(self):
        return str(self)


class gridStab:
    def __init__(self, n, pauli):
        self.pauli = pauli


def twoLoc(a, b, pgrd):
    flag = False
    c = a.merge(b)

    if len(c.index) == 2:
        return True
    for i in c.index:
        t = i // 2
        op = pauliLog(n, pgrd[t] * 4, [j % n for j in range(t * 2, t * 2 + 4)])
        tmp = c.merge(op)
        if len(tmp.index) == 2:
            return True
    return False


n = 6
m = n // 2


pgrd = "xz" * (m // 2) + "y" * (m % 2)
print(pgrd)

a = pauliLog(n, "XZ", [0, 8])
b = pauliLog(n, "xy", [6, 8])
a.fit(b, -1)

xzy


True

In [2]:
toSearch = [("x", i) for i in range(m)] + [("z", i) for i in range(m)]
stack = []
xglob = {}
zglob = {}


def softCons(logOp, ind):
    count = 0
    if ind != m and ind != 0:
        if not twoLoc(logOp, stack[ind - 1], pgrd):
            count += 1
    if ind == m - 1 or ind == 2 * m - 1:
        if not twoLoc(logOp, stack[ind - m + 1], pgrd):
            count += 1
    return count


def check(logOp, ind):
    toComp = [1 for _ in range(ind)]
    result = True
    if ind >= m:
        toComp[ind - m] = -1
    for i in range(ind):

        if not logOp.fit(stack[i], toComp[i]):

            result = False
            break
    # if ind == 5:
    #     print("close ", stack + [logOp])

    return result, softCons(logOp, ind)


def visited(tuple, ind):
    if ind >= m:
        return tuple in zglob.keys()
    else:
        return tuple in xglob.keys()


def search(ind, endureCount):

    # print(ind, [str(i) for i in stack])
    if ind >= 2 * m:
        return True
    if ind == 0:
        candInd = [0]
    else:
        candInd = list(range(m))
    log = None
    flag = False

    for i in candInd:
        p = ""
        for p1 in "xyz":
            if p1 != pgrd[i - 1] and p1 != pgrd[(i) % m]:
                p = p1 * 2
                candidate = [(p, (2 * i, 2 * i + 1))]
            elif p1 == pgrd[i - 1]:
                candidate = [
                    (p1 * 2, (2 * i, 2 * i + 1)),
                    (p1 + pgrd[(i + 1) % m], (2 * i, (2 * i + 2) % n)),
                    (p1 + pgrd[(i + 1) % m], (2 * i, (2 * i + 3) % n)),
                ]
            else:
                candidate = [
                    (p1 * 2, (2 * i, 2 * i + 1)),
                    (p1 + pgrd[(i - 2) % m], (2 * i, (2 * i - 2) % n)),
                    (p1 + pgrd[(i - 2) % m], (2 * i, (2 * i - 1) % n)),
                ]
            # print(f"candidate, i:{i}, ind:{ind}, p1:{p1}", candidate)
            # if ind == 2 and str(stack[-1]) == "('XZ', (0, 8))":
            #     print(i, pgrd, pgrd[i - 1], p1, candidate)
            for tup in candidate:
                log = pauliLog(n, tup[0], tup[1])
                # if visited(tup, ind):
                #     continue
                hardc, softc = check(log, ind)
                # if (
                #     ind == 2
                #     and str(log) == "('XY', (6, 8))"
                #     and str(stack[-1]) == "('XZ', (0, 8))"
                # ):
                #     print("check", hardc, softc)
                nUpper = endureCount - softc
                # if hardc and ind == 5 and tup == ("xy", (2, 5)):
                #     print(nUpper, softc)
                if hardc and nUpper >= 0:
                    stack.append(log)
                    if ind >= m:
                        zglob[tup] = True
                    else:
                        xglob[tup] = True
                    flag = flag or search(ind + 1, nUpper)
                    if flag:

                        return True
                    stack.pop(-1)


# stack = [('XX', (0, 1)), ('XZ', (0, 8)), ('XY', (6, 8)), ('XX', (4, 5)), ('XX', (2, 5))]
upper = 1
res = search(0, upper)
print(res)
print(stack)
print(pgrd)

True
[('XX', (0, 1)), ('XZ', (0, 5)), ('ZZ', (2, 3)), ('ZY', (2, 1)), ('YY', (0, 1)), ('YX', (4, 3))]
xzy


In [3]:
def strOp(op, n):
    temp = [" " for _ in range(n)]
    for k in op.map.keys():
        temp[k] = op.map[k]
    first = "[" + ",".join([temp[j] for j in range(0, n, 2)]) + "]"
    second = "[" + ",".join([temp[j] for j in range(1, n, 2)]) + "]"
    # print(first)
    # print(second)
    return first, second


for i in range(len(stack) // 2):

    print(f"X{i}\t\t\t\t\t Z{i}")

    op = stack[i]
    xfirst, xsecond = strOp(op, n)
    zfirst, zsecond = strOp(stack[i + m], n)
    print(xfirst, f"\t\t\t\t", zfirst)
    print(xsecond, f"\t\t\t\t", zsecond)

X0					 Z0
[X, , ] 				 [ ,Z, ]
[X, , ] 				 [Y, , ]
X1					 Z1
[X, , ] 				 [Y, , ]
[ , ,Z] 				 [Y, , ]
X2					 Z2
[ ,Z, ] 				 [ , ,Y]
[ ,Z, ] 				 [ ,X, ]


In [4]:
from Hamil_search import *
import numpy as np

In [5]:
n = 6
m = n // 2
Xlist = ["X0*X1", "-1*X2*X3"]
Zlist = ["2*Z2*Z3", "-2*Z4*Z5"]
Ylist = [f"-{int(2**(m-1))}*Y0*Y1", f"{int(2**(m-1))}*Y{n-2}*Y{n-1}"]
Hlist = Xlist + Ylist + Zlist
print(Hlist)
H = sum([pauliExpr2Mat(n, a) for a in Hlist])
config = {"target": 0, "distance": 2, "depth": 2, "thres": 1}
P = getProjector(n, H, config)
# Hinv = np.linalg.pinv(H)
space = getSpace(n, H, config)
print(space.shape[1])

['X0*X1', '-1*X2*X3', '-4*Y0*Y1', '4*Y4*Y5', '2*Z2*Z3', '-2*Z4*Z5']
8


0.9999999999999998
True
0.9999999999999998
True
0.9999999999999998
True
0.9999999999999998
True
0.9999999999999998
True
0.9999999999999998
True
0.9999999999999998
True
0.9999999999999998
True
(64, 8)


AttributeError: module pennylane has no attribute pauli_decompose

In [ ]:
from itertools import product
import pennylane as qml

a1 = [("00", "11"), ("00", "11"), ("00", "11")]
a2 = [("00", "11"), ("01", "10"), ("01", "-10")]
a3 = [("00", "-11"), ("00", "-11"), ("00", "-11")]
a4 = [("00", "-11"), ("01", "-10"), ("01", "10")]
a5 = [("01", "10"), ("00", "11"), ("00", "-11")]
a6 = [("01", "10"), ("01", "10"), ("01", "10")]
a7 = [("01", "-10"), ("00", "-11"), ("00", "11")]
a8 = [("01", "-10"), ("01", "-10"), ("01", "-10")]
ap = [a5, a6, a1, a2, a3, a4, a7, a8]
eff = [1, 1j, 1, 1j, 1j, -1, -1j, 1]


def vec(a):
    v1, v2, v3 = ket2Vec(2, a[0]), ket2Vec(2, a[1]), ket2Vec(2, a[2])
    return np.kron(np.kron(v1, v2), v3)


vecs = []
for i in range(len(ap)):
    a = ap[i]
    v = eff[i] * vec(a)
    print(LA.norm(v))
    result = H @ v - np.zeros(v.shape)
    vecs.append(v)
    print(LA.norm(result) < 1e-4)

U = np.column_stack(tuple(vecs))
print(U.shape)
HpenInverse = np.linalg.pinv(H)
Q = np.identity(P.shape[0]) - P
test = pauliExpr2Mat(n, "Y1*Z2+X3*Y4")
target = pauliExpr2Mat(n, "Y1*Z2*X3*Y4")
A = test @ HpenInverse @ test
Hlogi = -U.conj().T @ A @ U
res = qml.pauli_decompose(Hlogi, hide_identity=True, pauli=True)


def valid(V):
    P1 = target
    P2 = P @ V @ Q @ HpenInverse @ Q @ V @ P
    nonZ = np.nonzero(np.absolute(P1 - P2) > 1e-4)
    print(nonZ)
    return checkSame(P1, P2)


print(res)

AttributeError: module pennylane has no attribute pauli_decompose